In [165]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fashiondata/finalPredict.csv
/kaggle/input/fashiondata/finalTrain.csv


In [166]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Reshape
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D

from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.preprocessing.image import ImageDataGenerator

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from sklearn.model_selection import train_test_split

%matplotlib inline

In [167]:
Train= pd.read_csv('/kaggle/input/fashiondata/finalTrain.csv')
X_Test= pd.read_csv('/kaggle/input/fashiondata/finalPredict.csv')

In [168]:
X_Train = Train.drop(['label'],axis=1)/255
Y_Train = Train[['label']]

X_Train = X_Train.astype('float32')
X_Test = X_Test.astype('float32')/255

X_Train = X_Train.values.reshape(-1,28,28,1)
X_Test = X_Test.values.reshape(-1,28,28,1)

random_seed = 2
X_Train, X_Val, Y_Train, Y_Val = train_test_split(X_Train, Y_Train, test_size = 0.1, random_state=random_seed)

from keras.utils import to_categorical
num_classes = 10
Y_Train = to_categorical(Y_Train, num_classes)
Y_Val = to_categorical(Y_Val, num_classes)

In [169]:
def Fashion(input_shape):

    X_input = Input(input_shape)
    
    X = Conv2D(64, (5, 5), strides = (1, 1),padding='same', name = 'conv0')(X_input)    
    X = Conv2D(64, (5 ,5), strides = (1, 1),padding='same', name = 'conv1')(X_input)
    
    X = MaxPooling2D((2, 2), name='max_pool')(X)
    X = Dropout(0.25)(X)
    
    X = Conv2D(128, (3, 3), strides = (1, 1),padding='same', name = 'conv2')(X_input)
    X = Conv2D(128, (3, 3), strides = (1, 1),padding='same', name = 'conv3')(X_input)

    X = MaxPooling2D((2, 2), name='max_pool')(X)
    X = Dropout(0.25)(X)
    

    X = Flatten()(X)
    X = Dense(256, activation='relu', name='FC2')(X)
    X = Dropout(0.5)(X)
    X = Dense(10, activation='softmax', name='FC3')(X)

    model = Model(inputs = X_input, outputs = X, name='Digit')

    return model

In [170]:
Classifier = Fashion(X_Train.shape[1:])

In [171]:
Classifier.compile(optimizer = "adam" , loss = "categorical_crossentropy", metrics=["accuracy"])

In [172]:
Classifier.fit(x = X_Train, y = Y_Train, epochs = 100, batch_size = 128)

Epoch 1/30
422/422 [==============================] - 3s 7ms/step - loss: 0.4627 - accuracy: 0.8353
Epoch 2/30
422/422 [==============================] - 3s 6ms/step - loss: 0.3186 - accuracy: 0.8850
Epoch 3/30
422/422 [==============================] - 3s 6ms/step - loss: 0.2793 - accuracy: 0.8985
Epoch 4/30
422/422 [==============================] - 3s 6ms/step - loss: 0.2501 - accuracy: 0.9086
Epoch 5/30
422/422 [==============================] - 3s 6ms/step - loss: 0.2277 - accuracy: 0.9168
Epoch 6/30
422/422 [==============================] - 3s 6ms/step - loss: 0.2148 - accuracy: 0.9218
Epoch 7/30
422/422 [==============================] - 3s 6ms/step - loss: 0.2002 - accuracy: 0.9247
Epoch 8/30
422/422 [==============================] - 3s 6ms/step - loss: 0.1911 - accuracy: 0.9289
Epoch 9/30
422/422 [==============================] - 3s 6ms/step - loss: 0.1782 - accuracy: 0.9341
Epoch 10/30
422/422 [==============================] - 3s 6ms/step - loss: 0.1698 - accuracy: 0.9361

In [173]:
preds = Classifier.evaluate(x = X_Val, y = Y_Val)
print()
print ("Loss = " + str(preds[0]))
print ("Validation Set Accuracy = " + str(preds[1]))

188/188 [==============================] - 0s 2ms/step - loss: 0.3513 - accuracy: 0.9245

Loss = 0.35132089257240295
Validation Set Accuracy = 0.9244999885559082


In [103]:
y_prediction = Classifier.predict(X_Test)

In [ ]:
Predict = (np.argmax(y_prediction, axis=1)).reshape(-1, 1)

In [ ]:
Predictions = pd.DataFrame({'True Label': Predict[:, 0]})
Predictions.index.name = 'Index'
Predictions.index += 1.
Predictions.astype('int32').dtypes
print(Predictions)

In [ ]:
Image = X_Test[2,:,:,:]
Image = np.squeeze(Image)
plt.imshow(Image)

In [ ]:
Predictions.to_csv('teamB21Results.csv', index=False)